### EDA & Experiments

In [1]:
import pandas as pd
import emoji
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

c:\Users\valle\Documents\tugas-binus\semester-5\deep-learning\lab\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
try:
    df = pd.read_csv("../dataset/re_dataset.csv", encoding="utf-8")
    print("Success loading data")
except UnicodeDecodeError:
    print("Error in decoding")

Error in decoding


In [6]:
try:
    df = pd.read_csv("../dataset/re_dataset.csv", encoding="latin-1")
    print("Success loading data")
except UnicodeDecodeError:
    print("Error in decoding")

Success loading data


In [14]:
df

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13164,USER jangan asal ngomong ndasmu. congor lu yg ...,1,1,1,0,0,0,1,0,0,1,0,0
13165,USER Kasur mana enak kunyuk',0,1,0,0,0,0,0,0,0,0,0,0
13166,USER Hati hati bisu :( .g\n\nlagi bosan huft \...,0,0,0,0,0,0,0,0,0,0,0,0
13167,USER USER USER USER Bom yang real mudah terdet...,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
condition = (df['HS'] == 1) | (df['Abusive'] == 1)

# Filter the DataFrame and count the resulting rows
count_hs_or_abusive = df[condition].shape[0]

print("Result: ")
print(f"Total rows where 'HS' is 1 OR 'Abusive' is 1: {count_hs_or_abusive}")

Result: 
Total rows where 'HS' is 1 OR 'Abusive' is 1: 7309


In [16]:
df.count()

Tweet            13169
HS               13169
Abusive          13169
HS_Individual    13169
HS_Group         13169
HS_Religion      13169
HS_Race          13169
HS_Physical      13169
HS_Gender        13169
HS_Other         13169
HS_Weak          13169
HS_Moderate      13169
HS_Strong        13169
dtype: int64

In [17]:
count_hs_or_abusive = df[(df["HS"] == 1) | (df["Abusive"] == 1)].shape[0]
print(f"Found {len(df)}")
print(f"Found {count_hs_or_abusive} ({(count_hs_or_abusive / len(df) * 100):.2f}%) rows of Hate Speech or Abusive content.")
print(f"Found {len(df) - count_hs_or_abusive} ({((len(df) - count_hs_or_abusive) / len(df) * 100):.2f}%) rows of Neutral content.")

Found 13169
Found 7309 (55.50%) rows of Hate Speech or Abusive content.
Found 5860 (44.50%) rows of Neutral content.


Commonly found words: USER, RT, https links, encoded emojis

In [7]:
# Example of tweet containing emojis 
print(df['Tweet'][36])
print(df["Tweet"][103])

\xe2\x80\x9dPara Bajingan yang telah melukai Lady harus menerima ganjaran seratus kali lipat!!\xe2\x80\x9d(sanji) ##HBD17thPelautJKT48'
USER USER USER USER Wuih... cebong sewot...\n\xf0\x9f\x98\x82\xf0\x9f\x98\x82\xf0\x9f\x98\x82'


In [8]:
text = df['Tweet'][36]
text = text.replace("USER", "")
text = re.sub(r"^RT\s+", "", text)
text = re.sub(r"http\S+|www\.\S+", "", text)
text = re.sub(r"\\x\S*", "", text)
text = " ".join(text.split())

print("Normalized text:")
print(text)

Normalized text:
Bajingan yang telah melukai Lady harus menerima ganjaran seratus kali lipat!! ##HBD17thPelautJKT48'


In [2]:
class HateSpeechClassifier(nn.Module):
    def __init__(self, encoder: AutoModel, num_labels: int = 2):
        super().__init__()
        self.encoder = encoder
        hidden_size = self.encoder.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        cls_emb = outputs.last_hidden_state[:, 0, :]  # (batch, hidden)
        logits = self.classifier(cls_emb)

        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits, labels)

        return loss, logits

model_name = "indolem/indobert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)

model = HateSpeechClassifier(encoder=encoder)

print("Hidden size:", model.encoder.config.hidden_size)

c:\Users\valle\Documents\tugas-binus\semester-5\deep-learning\lab\myenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\valle\.cache\huggingface\hub\models--indolem--indobert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is n

Hidden size: 768


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
encoded = tokenizer(
    text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=128,
)

input_ids = encoded["input_ids"]
attention_mask = encoded["attention_mask"]

# Get embedding
model.eval()
with torch.no_grad():
    outputs = model.encoder(input_ids=input_ids, attention_mask=attention_mask)
    cls_emb = outputs.last_hidden_state[:, 0, :]  

print("CLS embedding shape:", cls_emb.shape)
print("First 10 dimensions:", cls_emb[0, :10])


CLS embedding shape: torch.Size([1, 768])
First 10 dimensions: tensor([-1.0927,  1.2645, -0.4416,  1.1606, -0.0378, -8.3703,  0.3743,  2.0603,
         0.0363,  0.1611])
